## Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import json

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

### Data Acquisition and Cleaning

Get the data from the Wikipedia website

In [3]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code

200

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [elem.text.strip() for elem in cols]
    data.append([elem for elem in cols if elem])
    
data[:5]

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

Load the raw data into a pandas data frame

In [6]:
columns = ['PostalCode', 'Borough', 'Neighborhood']
df_raw = pd.DataFrame(data[1:], columns=columns)
df_raw.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Clean the raw data and check for 'Not assigned' values

In [7]:
df = df_raw[df_raw['Borough'] != 'Not assigned'].reset_index(drop=True)
(df['Neighborhood'] == 'Not assigned').value_counts()

False    103
Name: Neighborhood, dtype: int64

In [8]:
df.shape

(103, 3)

### Assignment of Latitude and Longitude Coordinates

In [9]:
df_coordinates = pd.read_csv('data/geospatial_coordinates.csv')
df_coordinates.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_coordinates.head(1)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353


Join the data frames

In [11]:
df_combined = pd.merge(df, df_coordinates, on='PostalCode')
df_combined.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
df_combined.shape

(103, 5)

### Exploration and Clustering of Neighborhoods in Toronto

Create a map of Toronto with neighborhoods superimposed on top

In [13]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinates of Toronto are {latitude}, {longitude}.')

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighborhood in zip(df_combined['Latitude'], df_combined['Longitude'],
                                           df_combined['Borough'], df_combined['Neighborhood']):
    label = f'{latitude}, {longitude}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare credentials and version

In [15]:
CLIENT_ID = '*****'             # Foursquare ID
CLIENT_SECRET = '*****'         # Foursquare Secret
VERSION = '20180605'            # Foursquare API version
LIMIT = 100

Explore neighborhoods in Toronto

In [16]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # Make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # Return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            )
            for v in results
            ])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighborhood', 
        'Neighborhood Latitude', 
        'Neighborhood Longitude', 
        'Venue', 
        'Venue Latitude', 
        'Venue Longitude', 
        'Venue Category',
        ]
    
    return nearby_venues

In [17]:
toronto_venues = get_nearby_venues(names=df_combined['Neighborhood'],
                                   latitudes=df_combined['Latitude'],
                                   longitudes=df_combined['Longitude'])

KeyError: 'groups'

In [ ]:
toronto_venues.head(10)

Check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood')[['Venue']].count()

Find out how many unique categories can be curated from all the returned venues

In [ ]:
n_venues = len(toronto_venues['Venue Category'].unique())
print(f'There are {n_venues} unique categories.')

Prepare the data to analyze each neighborhood

In [ ]:
# One hot encoding
toronto_dummies = pd.get_dummies(toronto_venues['Venue Category'], prefix='_', prefix_sep='')

# Add neighborhood column back to dataframe
toronto_onehot = pd.concat([toronto_venues[['Neighborhood']], toronto_dummies], axis=1)

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

toronto_grouped.shape

In [ ]:
toronto_grouped[toronto_grouped['Neighborhood'] == 'Agincourt'].T.reset_index()

Print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print('---' + hood + '----')
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Put the data into a pandas data frame

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new data frame and display the top 10 venues for each neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(f'{ind+1}{indicators[ind]} Most Common Venue')
    except:
        columns.append(f'{ind+1}th Most Common Venue')

# Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :],
                                                                          num_top_venues)

neighborhoods_venues_sorted.head()

Run k-means to cluster the neighborhood into 5 clusters

In [ ]:
# Set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42)
kmeans.fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_

Create a new data frame that includes the cluster as well as the top 10 venues for each neighborhood

In [ ]:
# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_combined

# Merge toronto_grouped with df_combined to add latitude / longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'),
                                     on='Neighborhood', how='right')

toronto_merged.head()

Visualize the resulting clusters

In [ ]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7
        ).add_to(map_clusters)
       
map_clusters